In [1]:
import tensorflow as tf
import os
import numpy as np

def cifar_10_reshape(batch_arg):
    output=np.reshape(batch_arg,(10000,3,32,32)).transpose(0,2,3,1)
    return output

def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

def imgDataMultiflier(x, y):
    x0 = x
    x1 = tf.map_fn(lambda v: tf.image.flip_left_right(v), x0)
    x2 = tf.map_fn(lambda v: tf.image.random_hue(v, 0.3), x0)
    x3 = tf.map_fn(lambda v: tf.image.random_jpeg_quality(v, 30, 90), x0)
    x4 = tf.map_fn(lambda v: tf.image.random_hue(tf.image.flip_left_right(v), 0.3), x0)
    x5 = tf.map_fn(lambda v: tf.image.random_jpeg_quality(tf.image.flip_left_right(v), 30, 90), x0)
    x = tf.concat([x0, x1, x2, x3, x4, x5], axis=0)
    y = np.concatenate((y, y, y, y, y, y), axis=0)
    return x, y
totalDataNum = 300000

#Loading cifar-10 data and reshaping it to be batch_sizex32x32x3
batch1=unpickle('/Users/tglkwon/neovenezia/cifar/data_batch_1')
batch2=unpickle('/Users/tglkwon/neovenezia/cifar/data_batch_2')
batch3=unpickle('/Users/tglkwon/neovenezia/cifar/data_batch_3')
batch4=unpickle('/Users/tglkwon/neovenezia/cifar/data_batch_4')
batch5=unpickle('/Users/tglkwon/neovenezia/cifar/data_batch_5')

batch1_data=cifar_10_reshape(batch1[b'data'])
batch2_data=cifar_10_reshape(batch2[b'data'])
batch3_data=cifar_10_reshape(batch3[b'data'])
batch4_data=cifar_10_reshape(batch4[b'data'])
batch5_data=cifar_10_reshape(batch5[b'data'])

batch1_labels=batch1[b'labels']
batch2_labels=batch2[b'labels']
batch3_labels=batch3[b'labels']
batch4_labels=batch4[b'labels']
batch5_labels=batch5[b'labels']

test_batch=unpickle('/Users/tglkwon/neovenezia/cifar/test_batch')
test_images=cifar_10_reshape(test_batch[b'data'])
test_labels_data=test_batch[b'labels']
test_labels_one_hot = tf.one_hot(test_labels_data,10)

train_images=np.concatenate((batch1_data,batch2_data,batch3_data,batch4_data,batch5_data),axis=0)
train_labels_data=np.concatenate((batch1_labels,batch2_labels,batch3_labels,batch4_labels,batch5_labels),axis=0)
train_labels_one_hot = tf.one_hot(train_labels_data,10)

train_images, train_labels_data = imgDataMultiflier(train_images, train_labels_data)

train_images_grey = tf.image.rgb_to_grayscale(train_images)
train_images_reshape = tf.reshape(train_images_grey, [totalDataNum,32,32])
train_images_float = tf.div(tf.cast(train_images_reshape, tf.float32), 255.)

test_images_grey = tf.image.rgb_to_grayscale(test_images)
test_images_reshape = tf.reshape(test_images_grey, [10000,32,32])
test_images_float = tf.div(tf.cast(test_images_reshape, tf.float32), 255.)

In [2]:
#ith tf.device('/gpu:0'):
x = tf.placeholder(tf.float32, [None, 32, 32])
x_image = tf.reshape(x, [-1, 32, 32, 1])

# 첫번째 convolutional layer - 하나의 grayscale 이미지를 64개의 특징들(feature)으로 맵핑(maping)합니다.
W_conv1 = tf.Variable(tf.truncated_normal(shape=[5, 5, 1, 64], stddev=5e-2))
b_conv1 = tf.Variable(tf.constant(0.1, shape=[64]))
h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], padding='SAME') + b_conv1)

# 첫번째 Pooling layer
h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')

# 두번째 convolutional layer - 32개의 특징들(feature)을 64개의 특징들(feature)로 맵핑(maping)합니다.
W_conv2 = tf.Variable(tf.truncated_normal(shape=[5, 5, 64, 64], stddev=5e-2))
b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))
h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], padding='SAME') + b_conv2)

# 두번째 pooling layer.
h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')

# 세번째 convolutional layer
W_conv3 = tf.Variable(tf.truncated_normal(shape=[3, 3, 64, 128], stddev=5e-2))
b_conv3 = tf.Variable(tf.constant(0.1, shape=[128]))
h_conv3 = tf.nn.relu(tf.nn.conv2d(h_pool2, W_conv3, strides=[1, 1, 1, 1], padding='SAME') + b_conv3)

# 네번째 convolutional layer
W_conv4 = tf.Variable(tf.truncated_normal(shape=[3, 3, 128, 128], stddev=5e-2))
b_conv4 = tf.Variable(tf.constant(0.1, shape=[128])) 
h_conv4 = tf.nn.relu(tf.nn.conv2d(h_conv3, W_conv4, strides=[1, 1, 1, 1], padding='SAME') + b_conv4)

# 다섯번째 convolutional layer
W_conv5 = tf.Variable(tf.truncated_normal(shape=[3, 3, 128, 128], stddev=5e-2))
b_conv5 = tf.Variable(tf.constant(0.1, shape=[128]))
h_conv5 = tf.nn.relu(tf.nn.conv2d(h_conv4, W_conv5, strides=[1, 1, 1, 1], padding='SAME') + b_conv5)
h_conv5_flat = tf.reshape(h_conv5, [-1, 8*8*128])

w2 = tf.Variable(tf.truncated_normal(shape=[8 * 8 * 128, 384], stddev=5e-2))
b2 = tf.Variable(tf.constant(0.1, shape=[384]))
hidden = tf.nn.relu(tf.matmul(h_conv5_flat, w2)+b2)

keep_prob = tf.placeholder(tf.float32)
hidden_drop = tf.nn.dropout(hidden, keep_prob)

w0 =tf.Variable(tf.truncated_normal(shape=[384, 10], stddev=5e-2))
b0 = tf.Variable(tf.constant(0.1, shape=[10]))
k = tf.matmul(hidden_drop, w0)+b0
p = tf.nn.softmax(k)

y_ = tf.placeholder(tf.float32, [None, 10])
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=k,labels=y_)) 
train_step = tf.train.AdamOptimizer(0.001).minimize(loss)
correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess = tf.Session()
sess.run(tf.global_variables_initializer())

x_batch = np.array(train_images_float.eval(session=sess))
y_batch = np.array(tf.one_hot(train_labels_data, 10).eval(session=sess))
tx_batch = np.array(test_images_float.eval(session=sess))
ty_batch = np.array(tf.one_hot(test_labels_data, 10).eval(session=sess))    

In [ ]:
#tf.one_hot()
def labelTosoftmax(yy, classNum):
    y = np.zeros((classNum), dtype=int)
    y[yy]=1
    
    return y

In [3]:
import timeit

start = timeit.default_timer()
keep_prob_val = 0.8
#sess.run(tf.global_variables_initializer())

batchNum = 100
learningIteration = int(totalDataNum / batchNum)
for i in range(0, learningIteration):
    xi = i * batchNum
    x_batch_2 = np.asarray(x_batch[xi:xi+batchNum])
    y_batch_2 = np.asarray(y_batch[xi:xi+batchNum])    
    sess.run(train_step, feed_dict={x:x_batch_2, y_:y_batch_2, keep_prob:keep_prob_val})
'''
    y_batch_2 = []
    for yy in y_batch[xi:xi+batchNum]:
        y_batch_2.append(labelTosoftmax(yy, 10))
    x_batch_2 = np.asarray(x_batch[xi:xi+batchNum])
    y_batch_2 = np.asarray(y_batch_2)
    sess.run(train_step, feed_dict={x:x_batch_2, y_:y_batch_2, keep_prob:keep_prob_val})
'''
'''tx_batch = np.array(tf.Session().run(test_images_float))
ty_batch = np.array(test_labels_data)
ty_batch_2 = []
for yy in ty_batch:
    ty_batch_2.append(labelTosoftmax(yy, 10))
ty_batch_2 = np.asarray(ty_batch_2)
'''
loss_val, acc_val = sess.run([loss, accuracy], feed_dict={x:tx_batch, y_:ty_batch, keep_prob:keep_prob_val})

stop = timeit.default_timer()
processMin = (stop - start)/60
print ('Loss: %f, Accuracy: %f, Time: %f' % (loss_val, acc_val, processMin))


Loss: 0.986220, Accuracy: 0.676200, Time: 848.270323
